# Conda Meta recipe for `node` LTS, `npm` latest

In [37]:
import os
from hashlib import sha256
from requests import get
from urllib.parse import urljoin as join
import re
import yaml
import jinja2
from PIL import Image
from IPython.display import Markdown
from datetime import datetime

In [38]:
DIST = "https://nodejs.org/dist"

In [39]:
PLATFORMS = [
    ["linux", "64", "linux64"],
    ["linux", "86", "linux32"],
    ["darwin", "64", "osx"]
]

In [40]:
all_dists = get(DIST).content.decode()

In [41]:
LTS_V = sorted(re.findall(r"v4\.\d+\.\d+", all_dists))[-1]
LTS = LTS_V[1:]

In [42]:
shas_txt = get("/".join([DIST, LTS_V, "SHASUMS256.txt"])).content.decode().strip()

In [43]:
SHAS = dict([list(reversed(it.split(r"  "))) for it in shas_txt.split("\n")])

In [44]:
TMPL = """
{% macro fname(platform, arch) %}node-{{ LTS_V }}-{{ platform }}-x{{ arch }}.tar.gz{% endmacro %}
# generated by ../RECIPE.ipynb at {{ datetime.isoformat(datetime.now()) }}
package:
  name: nodejs
  version: {{ LTS }}

build:
  number: 0

about:
  home: https://nodejs.org
  license: MIT

source:
  {% for platform, arch, conda_platform in PLATFORMS %}
  fn: node-{{ LTS_V }}-{{ platform }}-x{{ arch }}.tar.gz # [linux64]
  url: {{ DIST }}/{{ LTS_V }}/{{ fname(platform, arch) }} # [{{ conda_platform }}]
  sha256: {{ SHAS[fname(platform, arch)] }} # [{{ conda_platform }}]
  {% endfor %}

requirements:
  build:
    - python # [win]

test:
  commands:
    - node --version
    - npm --version
    - npm config get prefix -g
    - npm install babel babel-core babel-preset-es2015 casperjs gulp istanbul jupyter-js-services karma less phantomjs
"""

In [45]:
with open(os.path.join("conda.recipe", "meta.yaml"), "w+") as meta:
    meta.write(jinja2.Template(TMPL).render(**globals()))

In [46]:
!git diff conda.recipe/meta.yaml

diff --git a/conda.recipe/meta.yaml b/conda.recipe/meta.yaml
index dc03fba..c1f86a1 100644
--- a/conda.recipe/meta.yaml
+++ b/conda.recipe/meta.yaml
@@ -1,6 +1,6 @@
 
 
-# generated by ../RECIPE.ipynb at 2016-02-09T11:45:45.252928
+# generated by ../RECIPE.ipynb at 2016-02-09T11:51:09.081250
 package:
   name: nodejs
   version: 4.2.6
